In [3]:
from collections import deque, namedtuple
from threading import Event, Lock, Thread
import random
import time

Pedido = namedtuple("Pedido", ["nombre", "mesa"])


class Gerente(Thread):

    def __init__(self, nombre: str, nombre_archivo: str, pedidos: deque,
                 lock_pedidos: Lock, lock_print: Lock, pedidos_tomados_evt: Event):
        super().__init__()
        self.nombre = nombre
        self.nombre_archivo = nombre_archivo
        self.pedidos = pedidos
        self.lock_pedidos = lock_pedidos
        self.lock_print = lock_print
        self.pedidos_tomados_evt = pedidos_tomados_evt

    def run(self):
        with open(self.nombre_archivo, encoding="utf-8") as archivo:
            archivo.readline().strip().split(",")
            for linea in archivo:
                pedido = Pedido(*linea.strip().split(","))
                with self.lock_print:
                    print(f"Gerente {self.nombre} anotando pedido de "
                        f"{pedido.nombre} para mesa {pedido.mesa}")
                with self.lock_pedidos:
                    self.pedidos.append(pedido)
                time.sleep(random.randint(1, 3))
        self.pedidos_tomados_evt.set()


class Cocinero(Thread):

    def __init__(self, nombre: str, pedidos: deque, platos: deque, lock_pedidos: Lock, lock_platos: Lock,
                 lock_print: Lock, pedidos_tomados_evt: Event, platos_cocinados_evt: Event):
        super().__init__()
        self.nombre = nombre
        self.platos = platos
        self.pedidos = pedidos
        self.lock_pedidos = lock_pedidos
        self.lock_platos = lock_platos
        self.lock_print = lock_print
        self.pedidos_tomados_evt = pedidos_tomados_evt
        self.platos_cocinados_evt = platos_cocinados_evt
        self.daemon = True

    def run(self):
        while True:
            with self.lock_pedidos:
                if len(self.pedidos) == 0:
                    if self.pedidos_tomados_evt.is_set():
                        self.platos_cocinados_evt.set()
                    continue
                time.sleep(random.randint(2, 5))
                pedido = self.pedidos.popleft()
                with self.lock_print:
                    print(f"Cociner@ {self.nombre} cocinando "
                        f"{pedido.nombre} para la mesa {pedido.mesa}")
                with self.lock_platos:
                    self.platos.append(pedido)
            time.sleep(random.randint(0, 1))


class Garzon(Thread):

    def __init__(self, nombre: str, platos: deque, lock_platos: Lock, lock_print: Lock,
                 platos_cocinados_evt: Event, platos_entregados_evt: Event):
        super().__init__()
        self.nombre = nombre
        self.platos = platos
        self.lock_platos = lock_platos
        self.lock_print = lock_print
        self.platos_cocinados_evt = platos_cocinados_evt
        self.platos_entregados_evt = platos_entregados_evt
        self.daemon = True

    def run(self):
        while True:
            with self.lock_platos:
                if len(self.platos) == 0:
                    if self.platos_cocinados_evt.is_set():
                        self.platos_entregados_evt.set()
                    continue
                time.sleep(random.randint(2, 5))
                plato = self.platos.popleft()
                with self.lock_print:
                    print(f"Garzón {self.nombre} entregando plato "
                        f"{plato.nombre} a mesa {plato.mesa}")
            time.sleep(random.randint(0, 1))


class Restoran:

    def __init__(self, nombre_restoran: str):
        self.nombre_restoran = nombre_restoran
        self.pedidos = deque()
        self.platos = deque()
        self.lock_pedidos = Lock()
        self.lock_platos = Lock()
        self.lock_print = Lock()
        self.pedidos_tomados_evt = Event()
        self.platos_cocinados_evt = Event()
        self.platos_entregados_evt = Event()
        self.garzones = []
        self.cocineros = []

    def abrir(self, nombre_archivo: str, nombre_gerente: str,
              nombres_garzones: list, nombres_cocineros: list):
        with self.lock_print:
            print(f"¡Buenos días! Ya abrimos el restorán {self.nombre_restoran}")
        self.gerente = Gerente(nombre_gerente, nombre_archivo, self.pedidos,
                               self.lock_pedidos, self.lock_print, self.pedidos_tomados_evt)
        self.gerente.start()
        for nombre in nombres_cocineros:
            cocinero = Cocinero(nombre, self.pedidos, self.platos,
                                self.lock_pedidos, self.lock_platos, self.lock_print,
                                self.pedidos_tomados_evt, self.platos_cocinados_evt)
            cocinero.start()
            self.cocineros.append(cocinero)
        for nombre in nombres_garzones:
            garzon = Garzon(nombre, self.platos, self.lock_platos, self.lock_print,
                            self.platos_cocinados_evt, self.platos_entregados_evt)
            garzon.start()
            self.garzones.append(garzon)
        self.platos_entregados_evt.wait()
        with self.lock_print:
            print(f"¡Buenas noches! Ya cerramos el restorán {self.nombre_restoran}")

In [4]:
garzones = ["Julián", "Clemente", "Diego", "Julio", "Carlos"]
cocineros = ["Hernán", "Dani", "Gatochico", "Dante", "Paqui"]
restoran = Restoran("DCChef")
restoran.abrir("data.csv", "Cata", garzones, cocineros)

¡Buenos días! Ya abrimos el restorán DCChef
Gerente Cata anotando pedido de papas fritas para mesa 1
Gerente Cata anotando pedido de hamburguesa con queso para mesa 2
Cociner@ Dani cocinando papas fritas para la mesa 1
Gerente Cata anotando pedido de chorrillana para mesa 3
Garzón Clemente entregando plato papas fritas a mesa 1
Cociner@ Dante cocinando hamburguesa con queso para la mesa 2
Cociner@ Dante cocinando chorrillana para la mesa 3
Garzón Julián entregando plato hamburguesa con queso a mesa 2
Gerente Cata anotando pedido de churrasco italiano para mesa 4
Gerente Cata anotando pedido de ensalada césar para mesa 5
Cociner@ Paqui cocinando churrasco italiano para la mesa 4
Garzón Clemente entregando plato chorrillana a mesa 3
Cociner@ Gatochico cocinando ensalada césar para la mesa 5
Gerente Cata anotando pedido de completo dinámico para mesa 6
Garzón Diego entregando plato churrasco italiano a mesa 4
Gerente Cata anotando pedido de congrio frito para mesa 7
Cociner@ Dante cocinan